In [51]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from geopy.distance import great_circle
from geopy.distance import geodesic

In [104]:
dta1 = pd.read_stata("metricsgame2.dta")
dta1 = dta1.dropna(subset = ["log_asian"])
dta1["zipcode"] = dta1["zipcode"].apply(lambda x:str(int(x)))
dta1

,zipcode,age_5,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,...,log_cat_4,log_cat_5,log_cat_6,log_cat_7,log_cat_8,log_cat_9,log_cat_10,log_cat_11,log_cat_12,log_cat_13
0,10001,622.0,893.0,832.0,1237.0,2524.0,2935.0,2747.0,1823.0,1682.0,...,0.905020,0.990223,0.320698,-0.500990,-2.283939,1.483363,0.900673,0.099499,-1.277134,0.128209
1,10002,2827.0,2907.0,3007.0,3209.0,5037.0,8004.0,6601.0,5552.0,4586.0,...,0.336127,0.127253,1.294309,-0.328032,-0.788214,2.195929,0.922130,1.558250,-1.145477,0.617439
2,10003,1846.0,797.0,593.0,7423.0,5037.0,8382.0,6007.0,4860.0,3274.0,...,1.162848,1.103298,0.077907,-1.298256,-1.418057,1.125690,0.436819,-0.260604,-2.075486,-0.861042
3,10004,100.0,83.0,111.0,135.0,152.0,412.0,454.0,367.0,240.0,...,1.792560,0.789258,NaN,NaN,-1.272165,-0.820180,0.114129,-2.524928,-1.138634,NaN
4,10005,401.0,268.0,126.0,244.0,1238.0,1949.0,1364.0,1363.0,623.0,...,1.935812,1.192750,NaN,NaN,NaN,0.391614,-0.452764,-0.494437,-0.301533,-3.245972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,11691,7040.0,5959.0,4339.0,4022.0,4403.0,5053.0,4178.0,4363.0,2714.0,...,-0.863698,0.863124,1.962091,-0.440428,-0.141050,2.102870,1.437689,1.815188,0.093654,1.296247
171,11692,1463.0,1743.0,1616.0,1494.0,1256.0,1273.0,1585.0,1277.0,1160.0,...,-0.174029,0.736842,1.871045,0.823551,-0.047277,1.465580,1.298526,1.766720,-0.087769,1.149527
172,11693,1080.0,941.0,654.0,645.0,897.0,788.0,631.0,873.0,857.0,...,-0.157241,0.736577,1.641439,1.079522,0.634997,1.964688,1.457737,2.314852,0.220053,1.119052
173,11694,1237.0,784.0,1218.0,1116.0,761.0,958.0,1455.0,1056.0,958.0,...,0.067661,1.645622,0.676354,1.017196,0.148786,1.350158,0.710256,1.907539,0.395037,0.625262


In [105]:
dta_c = pd.read_excel("church.xlsx")
dta_c.columns = ["name","lat","lon"]
merged_points = []
df = dta_c
merged_points = []

# merge near churches
for i in range(len(df)):
    merged_point = {'name': df['name'][i], 'lat': df['lat'][i], 'lon': df['lon'][i]}
    for j in range(i + 1, len(df)):
    
        distance = geodesic((df['lat'][i], df['lon'][i]), (df['lat'][j], df['lon'][j])).miles
        if distance < 1.3:
            
            merged_point['name'] += f"_{df['name'][j]}"
            merged_point['lat'] = (merged_point['lat'] + df['lat'][j]) / 2
            merged_point['lon'] = (merged_point['lon'] + df['lon'][j]) / 2


    if '_' in merged_point['name']:
        merged_points.append(merged_point)

dta_c = pd.DataFrame(merged_points)

# Build the time-series data relevant to tests

In [107]:
pattern1 = re.compile(r'^total_test.+?\d{4}$')
pattern2 = re.compile(r'^share_pos.+?\d{4}$')
pattern3 = re.compile(r'test.+?cap.+?\d{4}$')
total = [s for s in dta1.columns if pattern1.search(s)]
pos = [s for s in dta1.columns if pattern2.search(s)]
test_cap = [s for s in dta1.columns if pattern3.search(s)]

In [108]:
def helper1(ls_name,data,name):
    df1 = data[["zipcode"]+ls_name]
    ls_t = [s[-4:] for s in ls_name]
    df1.columns = ["zipcode"] + ls_t
    df1["zipcode"] = df1["zipcode"].apply(lambda x: str(int(x)))
    df1 = df1.transpose()
    df1.reset_index(inplace = True,drop=False)
    df1.columns = df1.iloc[0,:].astype(str).values
    df1.set_index("zipcode")
    df1 = df1.iloc[1:,:]
    df1.rename({"zipcode":"time"},axis = 1,inplace=True)
    df1 = pd.melt(df1, id_vars='time', var_name='zipcode', value_name=name)
    return df1
total_df = helper1(total,dta1,"total")
pos_df = helper1(pos,dta1,"pos")
test_cap_df = helper1(test_cap,dta1,"test_cap")

C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\1613421564.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["zipcode"] = df1["zipcode"].apply(lambda x: str(int(x)))
C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\1613421564.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["zipcode"] = df1["zipcode"].apply(lambda x: str(int(x)))
C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\1613421564.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [109]:
ts = pd.merge(left = total_df,right = pos_df,on = ["time","zipcode"],how = "outer",validate="1:1")
ts = pd.merge(left = ts,right = test_cap_df,on = ["time","zipcode"],how = "outer",validate="1:1")
ts

,time,zipcode,total,pos,test_cap
0,0609,10001,2373.0,0.155078,0.099094
1,0608,10001,2373.0,0.153814,0.099094
2,0607,10001,2335.0,0.156317,0.097507
3,0606,10001,2302.0,0.158558,0.096129
4,0605,10001,2267.0,0.161006,0.094667
...,...,...,...,...,...
11413,0407,11697,93.0,0.473118,0.025122
11414,0405,11697,80.0,0.425,0.02161
11415,0404,11697,75.0,0.413333,0.020259
11416,0403,11697,71.0,0.422535,0.019179


In [110]:
ts["L_pos"] = ts.groupby(["zipcode"])["pos"].shift(1)
ts["L_test_cap"] = ts.groupby(["zipcode"])["test_cap"].shift(1)
ts["L_total"] = ts.groupby(["zipcode"])["total"].shift(1)
ts

,time,zipcode,total,pos,test_cap,L_pos,L_test_cap,L_total
0,0609,10001,2373.0,0.155078,0.099094,NaN,NaN,NaN
1,0608,10001,2373.0,0.153814,0.099094,0.155078,0.099094,2373.0
2,0607,10001,2335.0,0.156317,0.097507,0.153814,0.099094,2373.0
3,0606,10001,2302.0,0.158558,0.096129,0.156317,0.097507,2335.0
4,0605,10001,2267.0,0.161006,0.094667,0.158558,0.096129,2302.0
...,...,...,...,...,...,...,...,...
11413,0407,11697,93.0,0.473118,0.025122,0.468085,0.025392,94.0
11414,0405,11697,80.0,0.425,0.02161,0.473118,0.025122,93.0
11415,0404,11697,75.0,0.413333,0.020259,0.425,0.02161,80.0
11416,0403,11697,71.0,0.422535,0.019179,0.413333,0.020259,75.0


# Build the cross-sectional data 

In [113]:
pattern = re.compile(r'\d{4}$')
left_col = [s for s in dta1.columns if not pattern.search(s)]
cs = dta1[left_col]
cs["zipcode"] = cs["zipcode"].apply(lambda x:str(x))

C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\3192606529.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs["zipcode"] = cs["zipcode"].apply(lambda x:str(x))


In [114]:
cs[["lat","lng"]]

,lat,lng
0,40.750629,-73.997177
1,40.715771,-73.986214
2,40.731831,-73.989182
3,40.688629,-74.018242
4,40.706032,-74.008827
...,...,...
170,40.601280,-73.761650
171,40.594090,-73.792892
172,40.590691,-73.809753
173,40.578270,-73.844757


In [115]:
cs['dis'] = None

# find the distance as IV
for index2, row2 in cs.iterrows():
    min_distance = float('inf')  
    for index1, row1 in dta_c.iterrows():
    
        distance = geodesic((row1['lat'], row1['lon']), (row2['lat'], row2['lng'])).kilometers
        
        if distance < min_distance:
            min_distance = distance
    
    cs.at[index2, 'dis'] = min_distance

C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\1230099990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs['dis'] = None


## cluster

In [118]:
def cal_center(data,lat,lon):
    
    latitudes = data[lat]
    longitudes = data[lon]


    center_latitude = np.mean(latitudes)
    center_longitude = np.mean(longitudes)

    center_coordinates = (center_latitude, center_longitude)
    distances_to_center = [great_circle(center_coordinates, (lat, lon)).kilometers for lat, lon in zip(latitudes, longitudes)]

    return distances_to_center
def cluster_geo(data,k_list):
    scaler = StandardScaler()
    X = scaler.fit_transform(data[['share_asian', 'share_black', 'share_hispanic', "d"]])

    k_values = k_list
    best_silhouette_score = -1
    best_k = None

    for k in k_list:
        kmeans = KMeans(n_clusters=k, random_state=42,n_init = "auto")
        labels = kmeans.fit_predict(X)
        silhouette_avg = silhouette_score(X, labels)
        
        print(f"Silhouette score for k={k}: {silhouette_avg}")
    
        if silhouette_avg > best_silhouette_score:
            best_silhouette_score = silhouette_avg
            best_k = k

    best_kmeans = KMeans(n_clusters=best_k, random_state=42)
    return best_kmeans.fit_predict(X).astype(int)

In [119]:
new_cs = pd.DataFrame()
for cat in np.unique(cs["borough_cat"].values):
    temp = cs.loc[cs["borough_cat"]==cat]
    print(temp.shape)
    temp["d"] = cal_center(temp,lat = "lat",lon = "lng")
    c = cluster_geo(temp,[2,3,4,5])
    temp["cluster"] = c.astype(str)
    temp["cluster"] = temp["cluster"].apply(lambda x: str(int(cat)) + str(x))
    new_cs = pd.concat([new_cs,temp],axis = 0)

(42, 156)
Silhouette score for k=2: 0.596565663560225
Silhouette score for k=3: 0.5550380051334045
Silhouette score for k=4: 0.61014495533538
Silhouette score for k=5: 0.5771686094578514
(24, 156)
Silhouette score for k=2: 0.6032390896575996
Silhouette score for k=3: 0.5009906987234364
Silhouette score for k=4: 0.5921541602117064
Silhouette score for k=5: 0.5430858726854232


C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\2749834198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["d"] = cal_center(temp,lat = "lat",lon = "lng")
d:\Miniconda\envs\bench_env\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\2749834198.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

(37, 156)
Silhouette score for k=2: 0.5498306337989922
Silhouette score for k=3: 0.5394850231076365
Silhouette score for k=4: 0.5174775112863681
Silhouette score for k=5: 0.5225132935393848
(58, 156)
Silhouette score for k=2: 0.5266577214337056
Silhouette score for k=3: 0.5810873761113122
Silhouette score for k=4: 0.506414717269938
Silhouette score for k=5: 0.5468330650753974
(12, 156)
Silhouette score for k=2: 0.6362759572908784
Silhouette score for k=3: 0.6361993638109998
Silhouette score for k=4: 0.5673438371607159
Silhouette score for k=5: 0.5060886593459445


C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\2749834198.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["cluster"] = c.astype(str)
C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\2749834198.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["cluster"] = temp["cluster"].apply(lambda x: str(int(cat)) + str(x))
C:\Users\Steven Sun\AppData\Local\Temp\ipykernel_1784\2749834198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [121]:
new_cs.to_excel("data_cs.xlsx")

# Merge

In [145]:
res = new_cs.merge(ts,on=["zipcode"],how = "left")

In [147]:
res["time1"] = res['time'].apply(lambda x: "2020-" + str(x)[:2] + "-" + str(x)[-2:])
res = res.sort_values(["time","zipcode"])

65       2020-04-01
131      2020-04-01
197      2020-04-01
263      2020-04-01
329      2020-04-01
            ...    
10296    2020-06-09
10362    2020-06-09
10428    2020-06-09
10494    2020-06-09
10560    2020-06-09
Name: time1, Length: 11418, dtype: object

In [148]:
pa = re.compile('^age.+?')
age = [s for s in dta1.columns if pa.search(s)]
res["population"] = res[age].sum(axis = 1)
# res.drop(["a","sum"],axis = 1,inplace=True)
res

,zipcode,age_5,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,...,total,pos,test_cap,L_pos,L_test_cap,L_total,a,sum,time1,population
65,10001,622.0,893.0,832.0,1237.0,2524.0,2935.0,2747.0,1823.0,1682.0,...,265.0,0.426415,0.011066,0.447368,0.012695,304.0,0.426415,NaN,2020-04-01,23947.0
131,10002,2827.0,2907.0,3007.0,3209.0,5037.0,8004.0,6601.0,5552.0,4586.0,...,542.0,0.461255,0.006955,0.486943,0.008354,651.0,0.461255,NaN,2020-04-01,77925.0
197,10003,1846.0,797.0,593.0,7423.0,5037.0,8382.0,6007.0,4860.0,3274.0,...,379.0,0.424802,0.006887,0.436073,0.007959,438.0,0.424802,NaN,2020-04-01,55035.0
263,10004,100.0,83.0,111.0,135.0,152.0,412.0,454.0,367.0,240.0,...,38.0,0.421053,0.012709,0.452381,0.014047,42.0,0.421053,NaN,2020-04-01,2990.0
329,10005,401.0,268.0,126.0,244.0,1238.0,1949.0,1364.0,1363.0,623.0,...,81.0,0.308642,0.009332,0.297872,0.010829,94.0,0.308642,NaN,2020-04-01,8680.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10296,11691,7040.0,5959.0,4339.0,4022.0,4403.0,5053.0,4178.0,4363.0,2714.0,...,8030.0,0.291407,0.122866,NaN,NaN,NaN,0.291407,NaN,2020-06-09,65356.0
10362,11692,1463.0,1743.0,1616.0,1494.0,1256.0,1273.0,1585.0,1277.0,1160.0,...,2246.0,0.269368,0.110957,NaN,NaN,NaN,0.269368,NaN,2020-06-09,20242.0
10428,11693,1080.0,941.0,654.0,645.0,897.0,788.0,631.0,873.0,857.0,...,1137.0,0.287599,0.089464,NaN,NaN,NaN,0.287599,NaN,2020-06-09,12709.0
10494,11694,1237.0,784.0,1218.0,1116.0,761.0,958.0,1455.0,1056.0,958.0,...,2514.0,0.280032,0.119697,NaN,NaN,NaN,0.280032,NaN,2020-06-09,21003.0


In [149]:
res.to_excel("data4.xlsx")

In [150]:
res.groupby("cluster")["zipcode"].count()

cluster
10     924
11     792
12     792
13     264
20     792
21     792
30    1056
31    1386
40    1716
41     330
42    1782
50     132
51     660
Name: zipcode, dtype: int64

In [142]:
res[["borough","borough_cat"]].drop_duplicates(keep = "last").sort_values("borough_cat")

,borough,borough_cat
2706,Manhattan,1.0
4290,Bronx,2.0
6732,Brooklyn,3.0
10560,Queens,4.0
11352,Staten Island,5.0
